In [19]:
import pandas as pd
import os
import sys
import json
import matplotlib.pyplot as plt

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)
from utils import *


from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score,cohen_kappa_score, accuracy_score
import pandas as pd
import json
import datasets

annotation_path = "/home/abdelrahman.sadallah/mbzuai/review_rewrite/inference/evalute_outputs/adapters/Llama-3.1-8B-Instruct/score_only/instruction/all/step-0/review_evaluation_human_annotation"

aspects = [ 'actionability', 'grounding_specificity', 'verifiability', 'helpfulness']

In [20]:
results = {}

for aspect in aspects:

    gold_labels_file = f"{annotation_path}/predictions_{aspect}_gold.jsonl"
    silver_labels_file = f"{annotation_path}/predictions_{aspect}_silver.jsonl"
    gold_labels = []
    silver_labels = []
    with open(gold_labels_file, 'r') as f:
        for line in f:
            gold_labels.append(json.loads(line)[f'{aspect}_label'])
    with open(silver_labels_file, 'r') as f:
        for line in f:
            silver_labels.append(json.loads(line)[f'{aspect}_label'])

    ds1 = datasets.load_dataset('boda/review_evaluation_human_annotation', name=aspect, split='gold').to_pandas()
    ds2 = datasets.load_dataset('boda/review_evaluation_human_annotation', name=aspect, split='silver').to_pandas()
    results[aspect] = pd.concat([ds1, ds2], ignore_index=True)

    results[aspect]['model_label'] = gold_labels + silver_labels

In [21]:
import json
confusion_matrices = {}
with open(f'outputs/human_best_model_agreement_results.txt', 'w') as f:
    for aspect, df in results.items():
        f.write(f'Agreement Statistics for {aspect}\n')
        f.write(f' Total number of samples: {len(df)}\n')
        ## get the prompt type
        
        

        
        gold_labels = []
        chatgpt_labels = []
        for index, row in df.iterrows():
            
            gold_label = row[f'{aspect}_label']
            chatgpt_label = row[f'model_label']
            chatgpt_label = str(chatgpt_label)
            if chatgpt_label.lower() in ['na','nan']:
                continue

            ## if the labels are numeric, convert them to string
            if str(gold_label).lower() not in ['x', 'no claim']:
                gold_label = str(int(float(gold_label)))
            else:
                gold_label = 'X'
            if str(chatgpt_label).lower() not in ['x', 'no claim']:
                ## extract the number from the string
                chatgpt_label = ''.join(filter(str.isdigit, chatgpt_label))


                chatgpt_label = str(int(float(chatgpt_label)))
            else:
                chatgpt_label = 'X'

            gold_labels.append(gold_label)
            chatgpt_labels.append(chatgpt_label)

        f.write(f' Agreement Statistics for {type}\n')
        ## for verifiability we have one mroe measure
        results = get_stats(gold=gold_labels, pred=chatgpt_labels, aspect=aspect)
        f.write(json.dumps(results, indent=4) + '\n')
        

        ## print confusion matrix
        cm = confusion_matrix(gold_labels, chatgpt_labels)
        labels_order = sorted(set(gold_labels) | set(chatgpt_labels))
        print(f'Labels order: {labels_order}')
        confusion_matrices[aspect] = cm

        f.write('-' * 50 + '\n')
            
    f.write('=' * 50 + '\n')
        


Labels order: ['1', '2', '3', '4', '5']
Labels order: ['1', '2', '3', '4', '5']
Labels order: ['1', '2', '3', '4', '5', 'X']
Labels order: ['1', '2', '3', '4', '5']
